# <center> Предсказание победителя в Dota 2
<center> <img src="https://meduza.io/impro/YnJZAHUW6WHz_JQm1uRPkTql_qAhbfxt3oFJLGH7CJg/fill/980/0/ce/1/aHR0cHM6Ly9tZWR1/emEuaW8vaW1hZ2Uv/YXR0YWNobWVudHMv/aW1hZ2VzLzAwNy8x/NTcvNjk1L29yaWdp/bmFsL0tMVThLbUti/ZG5pSzlibDA0Wmlw/WXcuanBn.webp" width="700" height="700">

[Почитать подбробнее](https://meduza.io/feature/2021/10/19/rossiyskaya-komanda-vyigrala-chempionat-mira-po-dota-2-i-poluchila-18-millionov-dollarov-postoyte-otkuda-takie-dengi-neuzheli-igrat-v-dotu-tak-slozhno)

#### [Оригинальная статья](https://arxiv.org/pdf/2106.01782.pdf)
    
### Начало

Посмотрим на готовые признаки и сделаем первую посылку. 

1. [Описание данных](#Описание-данных)
2. [Описание признаков](#Описание-признаков)
3. [Наша первая модель](#Наша-первая-модель)
4. [Посылка](#Посылка)

### Первые шаги на пути в датасайенс

5. [Кросс-валидация](#Кросс-валидация)
6. [Что есть в json файлах?](#Что-есть-в-json-файлах?)
7. [Feature engineering](#Feature-engineering)

### Импорты

In [1]:
import os
import json
import pandas as pd
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

%matplotlib inline

In [2]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные 
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [3]:
PATH_TO_DATA = "../input/bi-ml-competition-2023/"

df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                             "train_data.csv"), 
                                    index_col="match_id_hash")
df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                            "train_targets.csv"), 
                                   index_col="match_id_hash")

## <left>Наша первая модель

In [4]:
X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=SEED)

#### Обучим случайный лес

In [6]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
rf_model.fit(X_train, y_train)

CPU times: user 36.3 s, sys: 97.6 ms, total: 36.4 s
Wall time: 9.47 s


RandomForestClassifier(max_depth=7, n_estimators=300, n_jobs=-1,
                       random_state=10801)

In [7]:
%%time
from catboost import CatBoostClassifier
cb_model = CatBoostClassifier(verbose = False)
cb_model.fit(X_train, y_train)

CPU times: user 2min 6s, sys: 4.29 s, total: 2min 11s
Wall time: 35.4 s


#### Сделаем предсказания и оценим качество на отложенной части данных

In [8]:
rf_y_pred = rf_model.predict_proba(X_valid)[:, 1]
cb_y_pred = cb_model.predict_proba(X_valid)[:, 1]
rf_valid_score = roc_auc_score(y_valid, rf_y_pred)
cb_valid_score = roc_auc_score(y_valid, cb_y_pred)

rf_valid_score, cb_valid_score

(0.7754387258058622, 0.7987920804058042)

In [9]:
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ("Подумайте об установке ujson, чтобы работать с JSON объектами быстрее")
    
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ("Подумайте об установке tqdm, чтобы следить за прогрессом")

    
def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Возвращает записи о каждом матче
    """
    
    if n_matches_to_read is None:
        n_matches_to_read = total_matches
        
    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

In [10]:
from itertools import islice

def add_new_features(df_features, matches_file, start = None, end = None):
    """
    Аргуенты
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
            
    for match in islice(
        read_matches(matches_file),
        start,
        end
    ):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills
        
        for player, team, index in zip(match['players'], ['r']*5 + ['d']*5, list(range(1, 5 + 1)) + list(range(1, 5 + 1))):
            df_features.loc[match_id_hash, f"{team}{index}_kda"] = (player['kills'] + player['assists']) / (player['deaths'] + 1)
        

In [11]:
df_train_extended = df_train_features.copy()
add_new_features(df_train_extended, PATH_TO_DATA + 'train_raw_data.jsonl')

  0%|          | 0/31698 [00:00<?, ?it/s]

In [12]:
cv_scores_extended = cross_val_score(cb_model, df_train_extended.values, y, 
                                     cv=5, scoring="roc_auc", n_jobs=-1)

print(cv_scores_extended.mean())

0.803980728611751


In [13]:
df_test_extended = pd.read_csv(os.path.join(PATH_TO_DATA, "test_data.csv"), 
                                   index_col="match_id_hash")

add_new_features(df_test_extended, PATH_TO_DATA + "test_raw_data.jsonl")


  0%|          | 0/31698 [00:00<?, ?it/s]

In [14]:
from catboost import CatBoostClassifier
cb_model = CatBoostClassifier(verbose = False)
cb_model.fit(df_train_extended, y)

In [15]:
X_test = df_test_extended.values
y_test_pred = cb_model.predict(X_test)

df_submission = pd.DataFrame({"radiant_win": y_test_pred}, 
                                 index=df_test_extended.index)

submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

Файл посылки сохранен, как: submission_2023-04-08_14-15-52.csv
